In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from IPython.display import display


# Generate dummy data
np.random.seed(0)
n_obs = 25
weekly_electricity_data = pd.DataFrame({
    'Family_1_Electricity': np.random.normal(1000, 200, n_obs),
    'Family_2_Electricity': np.random.normal(900, 150, n_obs),
    'Family_3_Electricity': np.random.normal(1100, 250, n_obs),
    'Family_4_Electricity': np.random.normal(950, 180, n_obs),
    'Family_5_Electricity': np.random.normal(1050, 220, n_obs),
    'Temperature': np.random.normal(25, 5, n_obs),
    'Unemployment_Rate': np.random.normal(5, 2, n_obs),
    'Income': np.random.normal(5000, 1000, n_obs),  # New exogenous variable
    'Population': np.random.randint(1000, 5000, n_obs)  # New exogenous variable
})

# Split data into train and test sets
train_size = int(0.8 * len(weekly_electricity_data))
train_data = weekly_electricity_data[:train_size]
test_data = weekly_electricity_data[train_size:]

display(weekly_electricity_data)

ModuleNotFoundError: No module named 'statsmodels'

In [ ]:
# Function to perform ADF test and print results
def adf_test(series, name):
    result = adfuller(series)
    return pd.Series({
        'ADF Statistic': result[0],
        'p-value': result[1],
        'Critical Value (1%)': result[4]['1%'],
        'Critical Value (5%)': result[4]['5%'],
        'Critical Value (10%)': result[4]['10%']
    })

# Perform ADF test for each variable
adf_results = pd.DataFrame()
for col in train_data.columns:
    adf_results[col] = adf_test(train_data[col], col)

# Print ADF test results
print("ADF Test Results:")
print(adf_results)

# Visualize ADF test results
plt.figure(figsize=(10, 6))
for col in adf_results.index:
    plt.plot(adf_results.columns, adf_results.loc[col], marker='o', label=col)
plt.axhline(y=0.05, color='r', linestyle='--', label='Significance Level (5%)')
plt.title('ADF Test Results')
plt.xlabel('Variables')
plt.ylabel('ADF Statistic')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Tahap 1: Normalisasi Data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_train_data = scaler.fit_transform(train_data)
scaled_test_data = scaler.transform(test_data)

# Tahap 2: Penyetelan Parameter
p = 3  # Order of autoregressive
q = 3  # Order of moving average
k_exog = 4  # Number of exogenous variables

# Tahap 3: Penyetelan Koefisien
# Inisialisasi koefisien secara acak
np.random.seed(1)
phi = np.random.randn(p, len(train_data.columns), len(train_data.columns))  # Autoregressive coefficients
theta = np.random.randn(q, len(train_data.columns), len(train_data.columns))  # Moving average coefficients
beta = np.random.randn(len(train_data.columns), k_exog)  # Coefficients for exogenous variables

# Preallocate arrays to store forecasts
forecast = np.zeros((len(test_data), len(train_data.columns)))

# Inisialisasi lagged values
endog_lagged = np.zeros((p, len(train_data.columns)))
exog_lagged = np.zeros((p, k_exog))

# Loop untuk Prediksi
for t in range(len(scaled_test_data)):
    # Menghitung nilai lagged untuk variabel endogen
    endog_lagged[1, :] = endog_lagged[0, :]
    endog_lagged[0, :] = scaled_train_data[-1]
    
    # Menghitung nilai lagged untuk variabel eksogen
    exog_lagged[1, :] = exog_lagged[0, :]
    exog_lagged[0, :] = scaled_train_data[-1, -k_exog:]

    # Menghitung error residual
    residual = np.zeros(len(train_data.columns))
    if t >= p:
        for i in range(p):
            residual += phi[i] @ endog_lagged[i, :]
    
    # Melakukan prediksi menggunakan formula VARMA
    forecast[t] = residual + beta @ exog_lagged[0, :]

    # Memperbarui data latih dengan data uji yang diamati
    scaled_train_data = np.vstack([scaled_train_data, scaled_test_data[t]])

    # Log proses dan hasil setiap iterasi
    print(f"Iterasi ke-{t+1}: Prediksi {forecast[t]}")

# Tahap 4: Menampilkan Hasil Prediksi
# Invers transform hasil prediksi ke dalam skala aslinya
forecast = scaler.inverse_transform(forecast)
forecast_results = pd.DataFrame(forecast, index=test_data.index, columns=test_data.columns)

# Tampilkan hasil prediksi
print("\nForecast Results:")
display(forecast_results)


In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Menghitung MAE, MSE, dan RMSE untuk setiap keluarga
mae_values = []
mse_values = []
rmse_values = []

for family in ['Family_1', 'Family_2', 'Family_3', 'Family_4', 'Family_5']:
    actual_values = test_data[f'{family}_Electricity']
    predicted_values = forecast_results[f'{family}_Electricity']
    
    mae = mean_absolute_error(actual_values, predicted_values)
    mse = mean_squared_error(actual_values, predicted_values)
    rmse = np.sqrt(mse)
    
    mae_values.append(round(mae, 6))
    mse_values.append(round(mse, 6))
    rmse_values.append(round(rmse, 6))

# Menampilkan hasil evaluasi
evaluation_results = pd.DataFrame({
    'Family': ['Family_1', 'Family_2', 'Family_3', 'Family_4', 'Family_5'],
    'MAE': mae_values,
    'MSE': mse_values,
    'RMSE': rmse_values
})

print("Evaluation Results:")
display(evaluation_results)


In [ ]:
# Tahap 4: Visualisasi Hasil Prediksi

# Plot forecast vs actual
fig, axes = plt.subplots(nrows=5, ncols=1, figsize=(12, 15))

for i, family in enumerate(['Family_1', 'Family_2', 'Family_3', 'Family_4', 'Family_5']):
    axes[i].plot(test_data.index, test_data[f'{family}_Electricity'], label='Actual', color='blue')
    axes[i].plot(test_data.index, forecast_results[f'{family}_Electricity'], label='Forecast', color='red')
    axes[i].set_title(f'{family} Electricity Consumption Forecast')
    axes[i].set_xlabel('Time')
    axes[i].set_ylabel('Electricity Consumption')
    axes[i].legend()

plt.tight_layout()
plt.show()
